In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
dir = "/input/"

model_image_size = (320, 480)
fine_tune_layer = 172
final_layer = 314
visual_layer = 311
batch_size = 128

In [3]:
train_gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    rotation_range=10.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
)
gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
)

train_generator = train_gen.flow_from_directory(os.path.join(dir, 'train'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to train type {}".format(train_generator.class_indices))
valid_generator = gen.flow_from_directory(os.path.join(dir, 'valid'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to valid type {}".format(valid_generator.class_indices))

/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/keras_preprocessing/image.py:848: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 15506 images belonging to 10 classes.
subdior to train type {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}
Found 2644 images belonging to 10 classes.
subdior to valid type {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


In [4]:
input_tensor = Input((*model_image_size, 3))
x = input_tensor
x = Lambda(inception_v3.preprocess_input)(x)

base_model = InceptionV3(input_tensor=x, weights='imagenet', include_top=False)

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(base_model.input, x)

print("total layer count {}".format(len(base_model.layers)))

for i in range(fine_tune_layer):
    model.layers[i].trainable = False

total layer count 312


In [5]:
print("train_generator.samples = {}".format(train_generator.samples))
print("valid_generator.samples = {}".format(valid_generator.samples))
steps_train_sample = train_generator.samples // 128 + 1
steps_valid_sample = valid_generator.samples // 128 + 1

train_generator.samples = 15506
valid_generator.samples = 2644


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=4, validation_data=valid_generator, validation_steps=steps_valid_sample)

model.save("/data/inceptionV3h5")
print("model saved!")

Epoch 1/4


/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/keras_preprocessing/image.py:1131: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/keras_preprocessing/image.py:1139: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


122/122 [==============================] - 618s 5s/step - loss: 0.0622 - acc: 0.9809 - val_loss: 12.6246 - val_acc: 0.1475
Epoch 2/4
122/122 [==============================] - 596s 5s/step - loss: 0.0074 - acc: 0.9983 - val_loss: 1.6876 - val_acc: 0.5356
Epoch 3/4
122/122 [==============================] - 598s 5s/step - loss: 0.0019 - acc: 0.9995 - val_loss: 1.5602 - val_acc: 0.7806
Epoch 4/4
122/122 [==============================] - 597s 5s/step - loss: 5.5850e-04 - acc: 0.9999 - val_loss: 2.0816 - val_acc: 0.5514
model saved!


In [8]:
model.compile(optimizer=RMSprop(lr=1*0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1, validation_data=valid_generator, validation_steps=steps_valid_sample)

Epoch 1/1
122/122 [==============================] - 615s 5s/step - loss: 0.0013 - acc: 0.9997 - val_loss: 0.6732 - val_acc: 0.8476


In [9]:
model.save("/data/inceptionV3.h5")


In [13]:
model_image_size1 = 480
model_image_size2= 320

print("-------- loading test data")

for i in range(10):
    dir = os.path.join(basedir, "valid", "c%d"%i)
    image_files = glob.glob(os.path.join(dir,"*.jpg")) #返回F:\pose_recognication\data\train\ci文件夹中所有以jpg结尾的文件的路径
    print("loding {}, image count={}".format(dir, len(image_files)))
    
    exec ("X_test%s=[]"%i)
    exec ("y_test%s=[]"%i)
    for image_file in image_files:
        image = cv2.imread(image_file)
        exec ("X_test%s.append(cv2.resize(image, (model_image_size1, model_image_size2)))"%i)
        label = np.zeros(10, dtype=np.uint8)
        label[i]=1
        
        exec("y_test%s.append(label)"%i)
    exec ("X_test%s=np.array(X_test%s)"%(i,i))
    exec ("y_test%s=np.array(y_test%s)"%(i,i))

-------- loading test data
loding /input/valid/c0, image count=92
loding /input/valid/c1, image count=158
loding /input/valid/c2, image count=247
loding /input/valid/c3, image count=345
loding /input/valid/c4, image count=388
loding /input/valid/c5, image count=364
loding /input/valid/c6, image count=328
loding /input/valid/c7, image count=298
loding /input/valid/c8, image count=108
loding /input/valid/c9, image count=316


In [14]:
print("C0的准确率")
model.evaluate(X_test0,y_test0,verbose=1, )

C0的准确率
92/92 [==============================] - 2s 18ms/step


[0.03598606311108755, 1.0]

In [15]:
print("C1的准确率")
model.evaluate(X_test1,y_test1,verbose=1, )

C1的准确率
158/158 [==============================] - 1s 9ms/step


[0.0013762465471849788, 1.0]

In [16]:
print("C2的准确率")
model.evaluate(X_test2,y_test2,verbose=1, )

C2的准确率
247/247 [==============================] - 2s 7ms/step


[0.2743367628771284, 0.9271255070381319]

In [17]:
print("C3的准确率")
model.evaluate(X_test3,y_test3,verbose=1, )

C3的准确率
345/345 [==============================] - 2s 6ms/step


[0.5607127654379692, 0.834782608350118]

In [18]:
print("C3的准确率")
model.evaluate(X_test4,y_test4,verbose=1, )

C3的准确率
388/388 [==============================] - 2s 5ms/step


[1.8502314828105808, 0.21391752577319587]

In [19]:
print("C5的准确率")
model.evaluate(X_test5,y_test5,verbose=1, )

C5的准确率
364/364 [==============================] - 2s 6ms/step


[0.35699951583212547, 0.8763736263736264]

In [20]:
print("C6的准确率")
model.evaluate(X_test6,y_test6,verbose=1, )

C6的准确率
328/328 [==============================] - 2s 5ms/step


[1.4547130554896418e-06, 1.0]

In [21]:
print("C6的准确率")
model.evaluate(X_test7,y_test7,verbose=1, )

C6的准确率
298/298 [==============================] - 2s 6ms/step


[0.35577954881943313, 0.9966442953020134]

In [22]:
print("C6的准确率")
model.evaluate(X_test8,y_test8,verbose=1, )

C6的准确率
108/108 [==============================] - 1s 5ms/step


[0.0004006257729121932, 1.0]

In [23]:
print("C6的准确率")
model.evaluate(X_test9,y_test9,verbose=1, )

C6的准确率
316/316 [==============================] - 1s 5ms/step


[9.31127008607116, 0.0]